# OpenDSS

In [ ]:
import opendssdirect as odd

In [ ]:
odd.__version__

In [ ]:
odd.run_command("redirect ../../data/IEEE13/IEEE13Nodeckt.dss")

In [ ]:
odd.run_command("Solve")

In [ ]:
odd.Circuit.SetActiveClass("Vsource")

odd.Circuit.SetActiveElement("source")

In [ ]:
odd.CktElement.Name()

In [ ]:
BASEKV = odd.Vsources.BasekV()

In [ ]:
odd.Vsources.BasekV(110)

In [ ]:
odd.Vsources.BasekV()

In [ ]:
odd.Vsources.AngleDeg()

In [ ]:
odd.Vsources.AngleDeg(25)

In [ ]:
odd.Vsources.AngleDeg()

In [ ]:
# -*- coding: utf-8 -*-
import helics as h

fedinitstring = "--federates=1"
deltat = 0.01

helicsversion = h.helicsGetVersion()

### Create the federate Info object

In [ ]:
# Create Federate Info object that describes the federate properties */
print("Creating Federate Info")
fedinfo = h.helicsCreateFederateInfo()

# Set Federate name
print("Setting Federate Info Name")
h.helicsFederateInfoSetCoreName(fedinfo, "OpenDSSFederate")

# Set core type from string
print("Setting Federate Info Core Type")
h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")

# Federate init string
print("Setting Federate Info Init String")
h.helicsFederateInfoSetCoreInitString(fedinfo, fedinitstring)

# Set the message interval (timedelta) for federate. Note that
# HELICS minimum message time interval is 1 ns and by default
# it uses a time delta of 1 second. What is provided to the
# setTimedelta routine is a multiplier for the default timedelta.

# Set one second message interval
print("Setting Federate Info Time Delta")
h.helicsFederateInfoSetTimeProperty(fedinfo, h.helics_property_time_delta, deltat)

### Create the value federate

In [ ]:
vfed = h.helicsCreateValueFederate("OpenDSSFederate", fedinfo)
sub = h.helicsFederateRegisterSubscription(vfed, "voltage", "")
pub = h.helicsFederateRegisterGlobalTypePublication(vfed, "load", "complex", "")

### Enter execution

In [ ]:
h.helicsFederateEnterExecutingMode(vfed)

### Start simulation

In [ ]:
import numpy as np

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

In [ ]:
value = 0.0
prevtime = 0

granted_time = -1

for request_time in range(1, 300):

    while granted_time < request_time:
        granted_time = h.helicsFederateRequestTime(vfed, request_time)

    voltage_re, voltage_im = h.helicsInputGetComplex(sub)
    v, a = cart2pol(voltage_re, voltage_im)

    odd.Circuit.SetActiveClass("Vsource")
    odd.Circuit.SetActiveElement("source")
    odd.Vsources.AngleDeg(a)
    odd.Vsources.BasekV(v * BASEKV)
    print("Source voltage mag at time {}: {} ".format(request_time, v))
    print("Source voltage ang at time {}: {} ".format(request_time, a))
    odd.run_command("Solve")
    
    odd.Circuit.SetActiveClass("Line")
    odd.Circuit.SetActiveElement("650632")
    S = sum([complex(x,y) for x, y in zip(odd.CktElement.Powers()[0:6], odd.CktElement.Powers()[1:6])])
    Pg = S.real
    Qg = S.imag
    print("Active power at time {}: {} ".format(request_time, Pg))
    print("Reactive power at time {}: {} ".format(request_time, Qg))
    
    h.helicsPublicationPublishComplex(pub, Pg / 1000, Qg / 1000)
    
    print()


h.helicsFederateFinalize(vfed)

h.helicsFederateFree(vfed)
h.helicsCloseLibrary()
print("Federate finalized")